In [2]:
# Import packages
import xarray as xr
import numpy as np
import pandas as pd
import packaging
import matplotlib.pyplot as plt
import netCDF4
import math
import ipywidgets as widgets


# Define functions
def read_netcdf4(filename):
    '''Read in data from netcdf4, assign headers, set datatypes, perform unit conversions if necessary'''
    # read in netcdf4 data to xarray
    storms_data = xr.open_dataset(filename)
    
    # convert xarray dataarray to pandas dataframe
    storms_data = storms_data.RAINRATE.to_dataframe()
    
    # tidy data
    storms_data = storms_data.drop(['lat', 'lon'], axis=1)
    storms_data = storms_data.droplevel(['x', 'y'])
    storms_data['datetime'] = storms_data.index
    storms_data = storms_data.reset_index(drop = True)
    storms_data['datetime']= pd.to_datetime(storms_data['datetime'])
    storms_data = storms_data.rename(columns={"RAINRATE": "pr"})
    storms_data = storms_data.iloc[:,[1, 0]]
    # perform unit conversions if necessary
    # default units for netcdf4 are mm to 0.1 mm accuracy
    # storms_data.loc[ : , 'pr'] = storms_data.loc[ : , 'pr']

    return storms_data

def read_csv(filename):
    '''Read in data from csv, assign headers, set datatypes, perform unit conversions if necessary, and find time step for
       the dataset in minutes.'''
# read data from csv
    headers = ['datetime', 'pr']
    dtypes = {'col1': 'str', 'col2': 'float'}
    date_cols = ['datetime']
    storms_data = pd.read_csv(filename, dtype = dtypes, header=None, names = headers, skiprows = [0, 1], parse_dates = date_cols)
    
    # storms_data = pd.concat([storms_data, storms_new], ignore_index = True)
    
    # perform unit conversions
    storms_data.loc[ : , 'pr'] = storms_data.loc[ : , 'pr']
        
    return storms_data

def find_timestep(storms_data):
    '''Find time step of dataset in minutes by subtracting two datetimes'''
    # determine time step by subtracting one datetime from another
    TS = storms_data.iloc[1, 0] - storms_data.iloc[0, 0]
    # convert timedelta type to int type in minutes
    TS = TS.total_seconds() / 60
    return TS

def project_by_intensity(storms_modeled_past, storms_modeled_future, storms_obs, TS, subset):
    '''This is the meat of the process. It accepts model and historical data, finds Gumbel shape factors from each dataset, uses them to find
    the CDF value that corresponds with each precipitation intensity, calculates delta factors, and returns a time series of historical
    precipitation that has been projected into the future.'''
    
    # resample observed dataset to find hourly sums. This aligns the volumes from the hourly model data and the 5-min observed data
    resample = storms_obs.resample('60min', on = 'datetime').sum()
    # adjust index to include same number of entries as storms_obs
    resample = resample.loc[resample.index.repeat(60 / TS)].reset_index()
    # add resampled data to storms_obs
    storms_obs['pr_hourly'] = resample.pr
    
    # remove all dry times from datasets - this way only rainy periods are used to characterize the precipitation
    storms_modeled_past_no_0 = storms_modeled_past[storms_modeled_past['pr'] != 0]
    storms_modeled_future_no_0 = storms_modeled_future[storms_modeled_future['pr'] != 0]
    storms_obs_no_0 = storms_obs[storms_obs['pr_hourly'] != 0]
    
            
    # calculate statistics for each storm duration based on full rolling input of ~past~ data and find shape factors
    past_mean_pr = storms_modeled_past_no_0['pr'].mean()
    past_std_pr  = storms_modeled_past_no_0['pr'].std()
    euler_mascheroni_constant = 0.577215
    past_beta = past_std_pr * math.sqrt(6) / math.pi
    past_mu = past_mean_pr - past_beta * euler_mascheroni_constant

    # calculate statistics for each storm duration based on full rolling input of ~future~ data and find shape factors
    future_mean_pr = storms_modeled_future_no_0['pr'].mean()
    future_std_pr  = storms_modeled_future_no_0['pr'].std()
    future_beta = future_std_pr * math.sqrt(6) / math.pi
    future_mu = future_mean_pr - future_beta * euler_mascheroni_constant
        
    # calculate statistics for each storm duration based on full rolling input of ~observed~ data and find shape factors
    obs_mean_pr = storms_obs_no_0['pr_hourly'].mean()
    obs_std_pr  = storms_obs_no_0['pr_hourly'].std()
    obs_beta = obs_std_pr * math.sqrt(6) / math.pi
    obs_mu = obs_mean_pr - obs_beta * euler_mascheroni_constant
    
    
    # find the associated CDF value for each precipitation time step, solve for delta factors, and project the observed values into the future
    for count, value in enumerate(subset.loc[ : ,'pr']):
        subset.loc[count, 'obs_CDF'] = math.exp(-1 * math.exp(-1 * (value - float(obs_mu)) / float(obs_beta)))
        subset.loc[count, 'past_CDF'] = math.exp(-1 * math.exp(-1 * (value - float(past_mu)) / float(past_beta)))
        subset.loc[count, 'future_CDF'] = math.exp(-1 * math.exp(-1 * (value - float(future_mu)) / float(future_beta)))
        subset.loc[count, 'past_pr'] = -1 * past_beta * (math.log(-1 * math.log(subset.loc[count, 'obs_CDF']))) + past_mu
        subset.loc[count, 'future_pr'] = -1 * future_beta * (math.log(-1 * math.log(subset.loc[count, 'obs_CDF']))) + future_mu
        subset.loc[count, 'delta'] = subset.loc[count, 'future_pr'] / subset.loc[count, 'past_pr']
        subset.loc[count, 'projected_pr'] = subset.loc[count, 'delta'] * value
        
        
    # Add switch to fix edge case where model CDFs are negative for the smallest precipitation values
    # this is caused by the model under predicting precipitation for the study year
    subset.delta[subset.past_pr < 0] = 1
    subset.delta[subset.future_pr < 0] = 1
    subset.projected_pr[subset.delta == 1] = subset.pr

    return subset



In [ ]:
# Read netCDF4 data - climate model results
modeled_data = read_netcdf4('CONUS404-19791001-20221001-Dover-Air-Force-Base.nc')

# The following is used only while I don't have the full future CONUS404 dataset. It splits the past data in half 
# and pretends that the latter part is from the future
storms_modeled_past = modeled_data.iloc[:188471,:]
storms_modeled_future = modeled_data.iloc[188471:,:]
storms_modeled_future.reset_index(inplace = True, drop = True)

# Read full extent of observed data - tipping bucket rain gauge
# Names of all data files
files = ['files/dover_rain_gauge/DDFS.01-JAN-2005=01-JAN-2010.csv', 
         'files/dover_rain_gauge/DDFS.01-JAN-2010=01-JAN-2015.csv',
         'files/dover_rain_gauge/DDFS.01-JAN-2015=01-JAN-2020.csv',
         'files/dover_rain_gauge/DDFS.01-JAN-2020=01-NOV-2023.csv'
        ]

# Read each file individually
storms_obs0 = read_csv(files[0])
storms_obs1 = read_csv(files[1])
storms_obs2 = read_csv(files[2])
storms_obs3 = read_csv(files[3])

# Combine files into one dataframe
storms_obs = pd.concat([storms_obs0, storms_obs1, storms_obs2, storms_obs3], ignore_index = True)

# Replace all negative/error values with 0
# The data from this site has some error values because the tipping bucket rain gauge doesn't work well with snow
storms_obs.pr[storms_obs.pr < 0] = 0

# Read a time series which is to be projected into the future
data_2012 = read_csv('2012_historical.csv')

# Find the timestep of the observed data
TS = find_timestep(storms_obs)

# Project yime series and display results
projected_by_intensity = 0
projected_by_intensity = project_by_intensity(storms_modeled_past, storms_modeled_future, storms_obs, TS, data_2012)
display(projected_by_intensity)